In [7]:
import os, re, json
import pandas as pd
import networkx as nx
from pathlib import Path

GRAPHML = Path("graph_week6.graphml")
EDGELIST = Path("graph_week6.edgelist")
ENTITIES_CSV = Path("entities.csv")
RELATIONS_CSV = Path("relations.csv")

G = nx.Graph()

def load_from_graphml(p: Path):
    try:
        g = nx.read_graphml(p)
        GG = nx.Graph()
        for n, d in g.nodes(data=True):
            GG.add_node(str(n), **{k: d[k] for k in d})
        for u, v, d in g.edges(data=True):
            GG.add_edge(str(u), str(v), **{k: d[k] for k in d})
        return GG
    except Exception:
        return None

def load_from_edgelist(p: Path):
    try:
        g = nx.read_edgelist(str(p))
        return g
    except Exception:
        return None

def load_from_entities_relations(ents: Path, rels: Path):
    try:
        df_e = pd.read_csv(ents)
        df_r = pd.read_csv(rels)
        g = nx.Graph()
        for _, r in df_e.iterrows():
            g.add_node(str(r["entity"]), type=str(r.get("type", "")))
        for _, r in df_r.iterrows():
            g.add_edge(
                str(r["head"]), str(r["tail"]),
                doc_id=str(r.get("doc_id", "?")),
                sentence=str(r.get("sentence", "")),
            )
        return g
    except Exception:
        return None

loaded = None
if GRAPHML.exists():
    loaded = load_from_graphml(GRAPHML)
elif EDGELIST.exists():
    loaded = load_from_edgelist(EDGELIST)
elif ENTITIES_CSV.exists() and RELATIONS_CSV.exists():
    loaded = load_from_entities_relations(ENTITIES_CSV, RELATIONS_CSV)

if loaded is None:
    G = nx.Graph()
    nodes = [("Method X","METHOD"),("Author A","AUTHOR"),("Dataset D1","DATASET"),("Paper P3","PAPER"),("Metric F1","METRIC")]
    for n,t in nodes: G.add_node(n, type=t)
    G.add_edge("Method X","Author A",  doc_id="doc1", sentence="Method X was introduced by Author A.")
    G.add_edge("Method X","Dataset D1",doc_id="doc1", sentence="Method X was evaluated on Dataset D1 with F1=0.78.")
    G.add_edge("Method X","Paper P3",  doc_id="doc4", sentence="Paper P3 applies Method X to Dataset D2 and reports Accuracy 0.82.")
else:
    G = loaded

print(f"✅ Graph loaded: {len(G.nodes())} nodes, {len(G.edges())} edges")


✅ Graph loaded: 5 nodes, 3 edges


In [8]:
def node_type(n: str):
    t = G.nodes[n].get("type")
    if t: return t
    if re.search(r"^Method\s+[A-Z]", n):  return "METHOD"
    if re.search(r"^Author\s+[A-Z]", n):  return "AUTHOR"
    if re.search(r"^Dataset\s+[A-Z0-9]+", n): return "DATASET"
    if re.search(r"^(Paper|Survey)\s+[A-Z0-9]+", n): return "PAPER"
    if re.search(r"(F1|Accuracy|AUC)\b", n): return "METRIC"
    return "ENTITY"

def neighbors_with_evidence(n: str):
    out = []
    if n not in G: return out
    for u, v, data in G.edges(n, data=True):
        nb = v if u == n else u
        out.append({
            "neighbor": nb,
            "neighbor_type": node_type(nb),
            "doc_id": data.get("doc_id", "?"),
            "sentence": data.get("sentence", "")
        })
    return out

def detect_method_from_query(q: str):
    ql = (q or "").lower()
    for n in G.nodes():
        if node_type(n) == "METHOD" and n.lower() in ql:
            return n
    m = re.search(r"\bmethod\s+([a-z0-9]+)\b", ql)
    if m:
        last = m.group(1).upper()
        for n in G.nodes():
            if node_type(n) == "METHOD" and n.split()[-1].upper() == last:
                return n
    return None


In [9]:
def decompose(query: str):
    q = (query or "").lower()

    if "which author" in q and "which dataset" in q:
        return [
            "Who proposed the mentioned method?",
            "Which dataset was that method evaluated on?"
        ]

    if ("introduced" in q or "proposed" in q) and "method" in q and "dataset" in q:
        return [
            "Which paper or author introduced the method?",
            "Which dataset did that method/paper use for F1 or evaluation?"
        ]

    return [query]


In [10]:
def answer_subq(subq: str, memory: dict):
    method = memory.get("method") or detect_method_from_query(memory.get("root_query","")) or detect_method_from_query(subq)

    if ("author" in subq.lower() or "who proposed" in subq.lower()) and method:
        ev = neighbors_with_evidence(method)
        authors = [e for e in ev if e["neighbor_type"] == "AUTHOR"]
        if authors:
            ans = "; ".join(sorted({a["neighbor"] for a in authors}))
            return {"subq": subq, "answer": ans, "evidence": authors, "memory_update": {"author": ans, "method": method}}

    if ("which dataset" in subq.lower() or "dataset" in subq.lower()) and method:
        ev = neighbors_with_evidence(method)
        dsets = [e for e in ev if e["neighbor_type"] == "DATASET"]
        if dsets:
            ans = "; ".join(sorted({d["neighbor"] for d in dsets}))
            return {"subq": subq, "answer": ans, "evidence": dsets, "memory_update": {"dataset": ans, "method": method}}

    if method:
        ev = neighbors_with_evidence(method)
        ans = " ; ".join(sorted({f"{e['neighbor']} ({e['neighbor_type']})" for e in ev})) or "No direct evidence."
        return {"subq": subq, "answer": ans, "evidence": ev, "memory_update": {"method": method}}

    return {"subq": subq, "answer": "No evidence found in graph.", "evidence": [], "memory_update": {}}


In [11]:
def multi_hop(query: str):
    subs = decompose(query)
    memory = {"root_query": query}
    hops = []
    for s in subs:
        h = answer_subq(s, memory)
        hops.append(h)
        memory.update(h.get("memory_update", {}))

    final = " ; ".join(h["answer"] for h in hops if h["answer"])
    citations = sorted({
        ev.get("doc_id") for h in hops for ev in h.get("evidence", []) if ev.get("doc_id")
    })

    return {"query": query, "subqs": subs, "hops": hops, "final": final, "citations": citations, "memory": memory}

q1 = "Which author proposed Method X, and which dataset did they evaluate it on?"
out1 = multi_hop(q1)

q2 = "Which dataset did the paper that introduced Method X use for F1?"
out2 = multi_hop(q2)

out1, out2


({'query': 'Which author proposed Method X, and which dataset did they evaluate it on?',
  'subqs': ['Who proposed the mentioned method?',
   'Which dataset was that method evaluated on?'],
  'hops': [{'subq': 'Who proposed the mentioned method?',
    'answer': 'Author A',
    'evidence': [{'neighbor': 'Author A',
      'neighbor_type': 'AUTHOR',
      'doc_id': 'doc1',
      'sentence': 'Method X was introduced by Author A.'}],
    'memory_update': {'author': 'Author A', 'method': 'Method X'}},
   {'subq': 'Which dataset was that method evaluated on?',
    'answer': 'Dataset D1',
    'evidence': [{'neighbor': 'Dataset D1',
      'neighbor_type': 'DATASET',
      'doc_id': 'doc1',
      'sentence': 'Method X was evaluated on Dataset D1 with F1=0.78.'}],
    'memory_update': {'dataset': 'Dataset D1', 'method': 'Method X'}}],
  'final': 'Author A ; Dataset D1',
  'citations': ['doc1'],
  'memory': {'root_query': 'Which author proposed Method X, and which dataset did they evaluate it on?'

In [12]:
def print_trace(out):
    print("Query:", out["query"])
    print("Final Answer:", out["final"])
    print("Citations:", ", ".join(out["citations"]) if out["citations"] else "(none)")
    for i, h in enumerate(out["hops"], 1):
        print(f"\nHop {i}: {h['subq']}\n -> {h['answer']}")
        for ev in h.get("evidence", [])[:3]:
            print("    -", ev.get("doc_id","?"), ":", ev.get("sentence","")[:140])

print_trace(out1)
print_trace(out2)


Query: Which author proposed Method X, and which dataset did they evaluate it on?
Final Answer: Author A ; Dataset D1
Citations: doc1

Hop 1: Who proposed the mentioned method?
 -> Author A
    - doc1 : Method X was introduced by Author A.

Hop 2: Which dataset was that method evaluated on?
 -> Dataset D1
    - doc1 : Method X was evaluated on Dataset D1 with F1=0.78.
Query: Which dataset did the paper that introduced Method X use for F1?
Final Answer: Author A ; Dataset D1
Citations: doc1

Hop 1: Which paper or author introduced the method?
 -> Author A
    - doc1 : Method X was introduced by Author A.

Hop 2: Which dataset did that method/paper use for F1 or evaluation?
 -> Dataset D1
    - doc1 : Method X was evaluated on Dataset D1 with F1=0.78.


In [13]:
import datetime, sys, platform, json

cfg = {
  "timestamp": datetime.datetime.now().isoformat(timespec="seconds"),
  "graph": {"nodes": len(G.nodes()), "edges": len(G.edges())},
  "engine": {"strategy": "self-ask", "hop_limit": len(out1["hops"])},
  "artifacts_used": {
      "graphml": str(GRAPHML) if GRAPHML.exists() else None,
      "edgelist": str(EDGELIST) if EDGELIST.exists() else None,
      "entities_csv": str(ENTITIES_CSV) if ENTITIES_CSV.exists() else None,
      "relations_csv": str(RELATIONS_CSV) if RELATIONS_CSV.exists() else None,
  }
}
json.dump(cfg, open("trackB_multihop_run_config.json","w"), indent=2)
print("Saved: trackB_multihop_run_config.json")


Saved: trackB_multihop_run_config.json
